In [ ]:
# default_exp parser

# Parser
> This module handles the processing and parsing of the metadata and spectrum data from blocks

In [ ]:
#hide
import sys, os
from pathlib import Path

# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))

%load_ext autoreload
%autoreload 2 

In [ ]:
#export
# from multiprocessing import set_start_method, Pool
# try:
#     set_start_method("spawn")
# except RuntimeError:
#     pass
import os
from pathlib import Path
from typing import *
from datetime import datetime
from collections import defaultdict
from fastcore.basics import uniqueify, partialler, listify
from fastcore.utils import parallel
from fastcore.foundation import L
from rich.progress import track
from rfpye.blocks import *
from rfpye.utils import *
from rfpye.constants import *
from rfpye.cyparser import cy_extract_compressed
import pandas as pd
import numpy as np

path_type = Union[str, Any]
bin_val = Union[int, bytes]
bytes_encoded = Union[int, bytes]
datetime_object = datetime

In [ ]:
#exports
def parse_bin(
    bin_file, 
    bytes_header: int = BYTES_HEADER, 
    marker: bytes = ENDMARKER, 
    slice_=None,
    btypes: Iterable = MAIN_BLOCKS.keys()
):
    with open(bin_file, mode="rb") as bfile:
        # O primeiro bloco do arquivo é o cabeçalho e tem 36 bytes de tamanho.
        header = bfile.read(bytes_header)
        body = bfile.read()
    if slice_ is not None:
        assert (
            slice_.start >= bytes_header
        ), f"The start of your slice has to be >= {bytes_header}, you passed {slice_.start} "
        body = body[slice_]
    return {
        "file_version": bin2int(header[:4]),
        "string": bin2str(header[4:]),
        "blocks": classify_blocks(body.split(marker), btypes=btypes),
    }

## Processamento dos blocos
A função seguinte `parse_bin` recebe um arquivo `.bin` e mapeia os blocos contidos nele retornando um dicionário que tem como chave o tipo de bloco e os valores como uma lista com os blocos extraídos sequencialmente.

In [ ]:
#hide
# Python >=3.8 only
# def binary_iter(bin_file: path_type, marker: bytes = b'\x00UUUU', block_size: int = 4096) -> Iterator[bytes]:
#     """
#     str, bytes, int > bytes
#     :param bin_file: arquivo binario que contém os dados
#     :param marker: separador de blocos
#     :param block_size: tamanho em bytes que é "lido" por vez no arquivo, evitando problemas de memória
#     :return: bloco em formato binario
#     Gerador que fornece a partir de de um arquivo binário, um bloco binário por vez.

#     """
#     with open(bin_file, mode='rb') as bfile:
#         # O primeiro bloco do arquivo é o cabeçalho e tem 36 bytes de tamanho.
#         yield bfile.read(36)
#         # As demais partes podem prosseguir normalmente
#         current = b''
#         while block := bfile.read(block_size):
#             current += block
#             while (markerpos := current.find(marker)) > 0:
#                 yield current[:markerpos]
#                 current = current[markerpos + len(marker):]

A função a seguir mapeia o arquivo `.bin` nos devidos blocos

In [ ]:
#exports
def classify_blocks(blocks: list, btypes: Iterable = MAIN_BLOCKS.keys()) -> Mapping[Tuple, Tuple]:
    """Receives an iterable L with binary blocks and returns a defaultdict with a tuple (block types, thread_id) as keys and a list of the Class Blocks as values
    :param file: A string or pathlib.Path like path to a `.bin`file generated by CFRS - Logger
    :return: A Dictionary with block types as keys and a list of the Class Blocks available as values
    """
    map_block: Mapping[Tuple, L] = defaultdict(L)
    btypes = listify(btypes)
#     index = BYTES_HEADER
    for block in blocks:
        bloco_base = create_base_block(block)
        btype, btid = bloco_base.type, bloco_base.thread_id
#         stop = index + DATA_BLOCK_HEADER + len(bloco_base.data) + CHECKSUM
        if btype not in btypes:
#             index = stop + LEN_MARKER
            continue
        bloco = block_constructor(btype, bloco_base)
        if btype in SPECTRAL_BLOCKS:
            gerror = getattr(bloco, "gerror", -1)
            if gerror != -1 or bloco._level_len != bloco.ndata:
#                 index = stop + LEN_MARKER
                continue
        elif btype == GPS_BLOCK:
            if not getattr(bloco, 'gps_status'): continue # equals to zero

#         map_block[(btype, btid)].append(
#             ((index, stop), bloco)
#         )
        map_block[(btype, btid)].append(bloco)
#         index = stop + LEN_MARKER
    return map_block

In [ ]:
#exporti
def _extract_uncompressed(
    blocks: Iterable, rows: int, cols: int, min_level: float, dtype=np.float16
):
    levels = np.full((rows, cols), min_level, dtype=dtype)
    for b, block in enumerate(blocks):
        levels[b] = block.block_data
    return levels

In [ ]:
#exports
def extract_level(spectrum_blocks: L, dtype=np.float32) -> pd.DataFrame:
    """Receives a mapping `spectrum_blocks` and returns the Matrix with the Levels as values, Frequencies as columns and Block Number as index.
    :param pivoted: If False, optionally returns an unpivoted version of the Matrix
    """
    assert len(spectrum_blocks), f"The spectrum block list is empty"
    spectrum_blocks = spectrum_blocks.itemgot(1)
    block = spectrum_blocks[0]
    assert block.type in (63, 64, 67, 68), f"The input blocks are not spectral blocks"
    rows = len(spectrum_blocks)
    min_level = block.offset - 127.5
    if block.type in (63, 67):
        cols = block.ndata
        frequencies = getattr(block, "frequencies")
        return pd.DataFrame(
            _extract_uncompressed(spectrum_blocks, rows, cols, min_level, dtype),
            columns=frequencies,
        )
    else:
        cols = block.norig
        thresh = block.thresh - 1
        block_data = [b.block_data for b in spectrum_blocks]
        frequencies = np.linspace(block.start_mega, block.stop_mega, num=cols)
        levels = cy_extract_compressed(block_data, rows, cols, thresh, min_level)
        if dtype != np.float32:
            levels = levels.astype(dtype)
        return pd.DataFrame(levels, columns=frequencies)

In [ ]:
#export
def meta2df(meta_list: Iterable, optimize: bool = True) -> pd.DataFrame:
    """Receives and Iterable `metalist` with metadata and converts it to a DataFrame"""
    df = pd.DataFrame(meta_list)
    dt_features = ["wallclock_datetime"] if "wallclock_datetime" in df.columns else []
    if optimize:
        df = df_optimize(df, dt_features)
    if dt_features:
        df = df.set_index("wallclock_datetime")
    return df

In [ ]:
#export
def rowattrs(row, attrs):
    return  {
                **{"start_byte": row[0][0], "stop_byte": row[0][1]},
                **getattrs(row[1], attrs),
            }

In [ ]:
#export
def extract_metadata(blocos: L, attrs: list = None) -> pd.DataFrame:
    """Receives a list of blocks, extracts the metadata from them and return a DataFrame"""
    func = partialler(getattrs, attrs=attrs)
    df = meta2df(blocos.map(func))
    return df

In [ ]:
#exporti
def _export_meta(
    parsed_blocks: tuple,
    filter_attrs: Union[None, dict] = None,
    save: bool = False,
    stem: Union[str, Path] = None,
    saida: Union[str, Path] = None,
    ext: str = ".csv",
) -> Union[None, pd.DataFrame]:

    (tipo, tid), blocos = parsed_blocks
    if filter_attrs is None:
        attrs = None
    elif isinstance(filter_attrs, dict):
        attrs = filter_attrs.get(tipo)
    else:
        raise ValueError(f"Formato desconhecido do argumento {dict_attrs}:{type(dict_attrs)}, é esperado um dicionário")
    meta = extract_metadata(blocos, attrs)
    if save:
        dt_features = isinstance(meta.index, pd.DatetimeIndex)
        name = f"{stem}-B_{tipo}_TId_{tid}"
        if ext == ".csv":
            meta.to_csv(Path(saida) / f"{name}{ext}", index=bool(dt_features))
        elif ext == ".xlsx":
            meta.to_excel(Path(saida) / f"{name}{ext}", index=bool(dt_features))
        elif ext == ".fth":
            meta.to_feather(Path(saida) / f"{name}{ext}")
        else:
            raise ValueError(f"Extension {ext} not implemented")
    return tipo, meta

In [ ]:
#exports
def export_metadata(
    parsed_bin: dict, 
    filter_attrs: Union[None, dict] = None,
    save: bool = False, 
    stem: Union[str, Path] = None, 
    saida: Union[str, Path] = None, 
    ext: str = ".csv"
) -> None:

    file_version, string, blocks = parsed_bin.values()
    func = partialler(_export_meta, filter_attrs=filter_attrs, save=save, stem=stem, saida=saida, ext=ext)
    func.__module__ = _export_meta.__module__
    metas = parallel(func, list(blocks.items()), threadpool=True, n_workers=os.cpu_count())
    output = defaultdict(list)
    for t,m in metas: output[t].append(m)    
    return {k:pd.concat(v) for k,v in output.items()}

In [ ]:
#exporti
def _export_level(
    parsed_blocks: tuple,
    stem: Union[str, Path],
    saida: Union[str, Path],
    ext: str = ".fth",
    dtype: Union[str, np.dtype] = np.float16,
) -> Union[None, pd.DataFrame]:

    ((tipo, tid), blocos), index = parsed_blocks
    assert (
        tipo in SPECTRAL_BLOCKS
    ), "Tentativa de extrair espectro de um bloco que não é espectral"

    saida = Path(saida)
    level = extract_level(blocos, dtype)
    if index is not None:
        level.index = index
    dt_features = isinstance(level.index, pd.DatetimeIndex)

    name = f"{stem}-B_{tipo}_TId_{tid}"
    if ext == ".fth":
        if index is not None:
            level = level.reset_index()
        level.columns = [str(c) for c in level.columns]
        level.to_feather(f"{saida}/{name}{ext}")
    else:
        raise ValueError(f"Extension {ext} not implemented")

In [ ]:
#exports 
def export_level(
    stem: Union[str, Path],
    parsed_bin: dict,
    saida: Union[str, Path],
    ext: str = ".fth",
    index: pd.DatetimeIndex = None,
    dtype: Union[str, np.dtype] = np.float16,
) -> None:

    _, _, blocks = parsed_bin.values()
    blocks = [((t, i), b) for (t, i), b in blocks.items() if t in SPECTRAL_BLOCKS]
    if not index:
        index = [None] * len(blocks)
    items = list(zip(blocks, index))
    func = partialler(_export_level, stem=stem, saida=saida, ext=ext, dtype=dtype)
    func.__module__ = _export_meta.__module__
    parallel(func, items, n_workers=os.cpu_count(), pause=0.1)

In [ ]:
#hide
from nbdev.export import notebook2script; notebook2script()

Converted 00_filter.ipynb.
Converted 01_parser.ipynb.
Converted 02_utils.ipynb.
Converted 03_blocks.ipynb.
Converted 04_constants.ipynb.
Converted index.ipynb.
